In [1]:
import fitz  # PyMuPDF

def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    
    # Loop through each page
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text("text")  # Extract raw text

    return text

# Example usage
pdf_path = './documents/Lecture 02- Types of ML.pdf'
extracted_text = extract_text_from_pdf(pdf_path)
print(extracted_text)  # Print the first 1000 characters of the extracted text

Types of Machine 
Learning 
Types of 
Machine 
Learning
Types of 
Machine 
Learning
CAR
Types of 
Learning
Supervised (inductive) learning
• Given: training data + desired outputs
(labels)
Unsupervised learning
• Given:
training
data
(without
desired
outputs)
Semi-supervised learning
• Given: training data + a few desired outputs
Types of 
Machine 
Learning
Reinforcement learning
• A reinforcement learning algorithm, or
agent,
learns
by
interacting
with
its
environment.
• The agent receives rewards by performing
correctly
and
penalties
for
performing
incorrectly.
• The agent learns without intervention from
a human by maximizing its reward and
minimizing its penalty.
Supervised 
Learning
We are given input samples (X) and output
samples (y) of a function y = f(X).
We can represent the entire data set:
Data= (X,y); {Standard Notation}
We would like to “learn” f, and evaluate it
on new data. Types:
–Classification: y is factors (class labels).
–Regression: y is continuous, e.g. linear
re

In [2]:
import re

def clean_text(text):
    # Remove unwanted characters (e.g., page numbers, headers, footers)
    text = re.sub(r'\n+', ' ', text)  # Replace newlines with spaces
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)  # Remove non-ASCII characters
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = text.strip()  # Remove leading and trailing spaces
    return text

# Example usage
cleaned_text = clean_text(extracted_text)
print(cleaned_text[:1000])

Types of Machine Learning Types of Machine Learning Types of Machine Learning CAR Types of Learning Supervised (inductive) learning Given: training data + desired outputs (labels) Unsupervised learning Given: training data (without desired outputs) Semi-supervised learning Given: training data + a few desired outputs Types of Machine Learning Reinforcement learning A reinforcement learning algorithm, or agent, learns by interacting with its environment. The agent receives rewards by performing correctly and penalties for performing incorrectly. The agent learns without intervention from a human by maximizing its reward and minimizing its penalty. Supervised Learning We are given input samples (X) and output samples (y) of a function y = f(X). We can represent the entire data set: Data= (X,y); {Standard Notation} We would like to learn f, and evaluate it on new data. Types: Classification: y is factors (class labels). Regression: y is continuous, e.g. linear regression. Housing Price Pr

from transformers import AutoModelForCausalLM

# Load pre-trained GPT-2 model
model = AutoModelForCausalLM.from_pretrained('gpt2')

from transformers import AutoTokenizer

# Load a pre-trained tokenizer (e.g., for GPT-2)
tokenizer = AutoTokenizer.from_pretrained('gpt2')

# Set the pad token to be the same as the eos token
tokenizer.pad_token = tokenizer.eos_token  # Use eos_token as pad_token

model.save_pretrained('./gpt2_local')
tokenizer.save_pretrained('./gpt2_local')

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load the model and tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained('./gpt2_local')
model = AutoModelForCausalLM.from_pretrained('./gpt2_local')

In [32]:
import nltk

# Example large text (your cleaned_text)
large_text = cleaned_text  # Replace with your 12,113-word text

# Split text into sentences using nltk
#nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(large_text)

# Function to chunk text into pieces of max token length
def chunk_text(sentences, tokenizer, max_length):
    chunks = []  # To store final chunks
    current_chunk = []  # Collect sentences for the current chunk
    current_length = 0  # Track token count of the current chunk

    for sentence in sentences:
        tokenized_sentence = tokenizer(sentence, truncation=False, return_tensors="pt")
        sentence_length = tokenized_sentence['input_ids'].size(1)

        # If a sentence exceeds max_length, split it into smaller chunks
        while sentence_length > max_length:
            # Tokenize and truncate to max_length
            tokenized_chunk = tokenizer(sentence, truncation=True, max_length=max_length, return_tensors="pt")
            decoded_chunk = tokenizer.decode(tokenized_chunk['input_ids'][0], skip_special_tokens=True)
            chunks.append(decoded_chunk.strip())
            
            # Update the sentence to exclude the processed chunk
            sentence = sentence[len(decoded_chunk):].strip()
            tokenized_sentence = tokenizer(sentence, truncation=False, return_tensors="pt")
            sentence_length = tokenized_sentence['input_ids'].size(1)

        # If adding the current sentence exceeds max_length, finalize the current chunk
        if current_length + sentence_length > max_length:
            if current_chunk:  # Add only if the chunk isn't empty
                chunks.append(" ".join(current_chunk).strip())
            current_chunk = [sentence]  # Start a new chunk with the current sentence
            current_length = sentence_length  # Reset token count
        else:
            # Add sentence to the current chunk
            current_chunk.append(sentence)
            current_length += sentence_length

    # Add the final chunk if it exists
    if current_chunk:
        chunks.append(" ".join(current_chunk).strip())

    return chunks

# Create chunks with max_length=512
max_length = 30
chunks = chunk_text(sentences, tokenizer, max_length)

print(f"Total Chunks: {len(chunks)}")

Total Chunks: 10


In [33]:
# Print each chunk and its token count
for i, chunk in enumerate(chunks):
    tokenized = tokenizer(chunk, return_tensors="pt", truncation=False)
    print(f"Chunk {i+1}: {len(chunk.split())} words, {tokenized['input_ids'].size(1)} tokens")


Chunk 1: 25 words, 30 tokens
Chunk 2: 18 words, 30 tokens
Chunk 3: 19 words, 24 tokens
Chunk 4: 28 words, 30 tokens
Chunk 5: 20 words, 30 tokens
Chunk 6: 21 words, 30 tokens
Chunk 7: 14 words, 25 tokens
Chunk 8: 21 words, 30 tokens
Chunk 9: 12 words, 17 tokens
Chunk 10: 14 words, 25 tokens


In [35]:
sentences

['Types of Machine Learning Types of Machine Learning Types of Machine Learning CAR Types of Learning Supervised (inductive) learning Given: training data + desired outputs (labels) Unsupervised learning Given: training data (without desired outputs) Semi-supervised learning Given: training data + a few desired outputs Types of Machine Learning Reinforcement learning A reinforcement learning algorithm, or agent, learns by interacting with its environment.',
 'The agent receives rewards by performing correctly and penalties for performing incorrectly.',
 'The agent learns without intervention from a human by maximizing its reward and minimizing its penalty.',
 'Supervised Learning We are given input samples (X) and output samples (y) of a function y = f(X).',
 'We can represent the entire data set: Data= (X,y); {Standard Notation} We would like to learn f, and evaluate it on new data.',
 'Types: Classification: y is factors (class labels).',
 'Regression: y is continuous, e.g.',
 'linea

In [36]:
chunks

['Types of Machine Learning Types of Machine Learning Types of Machine Learning CAR Types of Learning Supervised (inductive) learning Given: training data + desired outputs',
 '(labels) Unsupervised learning Given: training data (without desired outputs) Semi-supervised learning Given: training data + a few desired',
 'outputs Types of Machine Learning Reinforcement learning A reinforcement learning algorithm, or agent, learns by interacting with its environment.',
 'The agent receives rewards by performing correctly and penalties for performing incorrectly. The agent learns without intervention from a human by maximizing its reward and minimizing its penalty.',
 'We can represent the entire data set: Data= (X,y); {Standard Notation} We would like to learn f, and evaluate it',
 'Supervised Learning We are given input samples (X) and output samples (y) of a function y = f(X). on new data.',
 'Types: Classification: y is factors (class labels). Regression: y is continuous, e.g. linear re

In [37]:
from torch.utils.data import Dataset

class LectureNotesDataset(Dataset):
    def __init__(self, texts, tokenizer, max_length):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        tokenized = self.tokenizer(
            self.texts[idx], 
            truncation=True, 
            padding='max_length', 
            max_length=self.max_length, 
            return_tensors='pt'
        )
        return {
            'input_ids': tokenized['input_ids'].squeeze(0),
            'attention_mask': tokenized['attention_mask'].squeeze(0),
            'labels': tokenized['input_ids'].squeeze(0)  # Labels are the same as input_ids
        }


dataset = LectureNotesDataset(chunks, tokenizer, max_length=512)

In [38]:
from torch.utils.data import DataLoader

data_loader = DataLoader(dataset, batch_size=2, shuffle=False)

In [39]:
from transformers import AdamW

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(50): 
    for batch in data_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(
            input_ids=batch['input_ids'], 
            attention_mask=batch['attention_mask'], 
            labels=batch['labels']  # Include labels to compute loss
        )
        
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        print(f"Loss: {loss.item()}")

c:\Users\Bim\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Loss: 8.659814834594727
Loss: 5.747227191925049
Loss: 2.448028564453125
Loss: 1.3296897411346436
Loss: 0.6615678071975708
Loss: 0.3584236204624176
Loss: 0.3228161036968231
Loss: 0.3476089835166931
Loss: 0.4093385338783264
Loss: 0.29808083176612854
Loss: 0.49591994285583496
Loss: 0.34673330187797546
Loss: 0.40733399987220764
Loss: 0.45797669887542725
Loss: 0.35465264320373535
Loss: 0.48873254656791687
Loss: 0.3520383834838867
Loss: 0.4194641709327698
Loss: 0.4736669361591339
Loss: 0.3265528678894043
Loss: 0.4130624532699585
Loss: 0.296543687582016
Loss: 0.35139966011047363
Loss: 0.35396167635917664
Loss: 0.27065566182136536
Loss: 0.3183683454990387
Loss: 0.2578621506690979
Loss: 0.3091924488544464
Loss: 0.2918037176132202
Loss: 0.24887317419052124
Loss: 0.29214292764663696
Loss: 0.24549004435539246
Loss: 0.26764562726020813
Loss: 0.28636249899864197
Loss: 0.22785261273384094
Loss: 0.28072589635849
Loss: 0.23573854565620422
Loss: 0.25528615713119507
Loss: 0.26684319972991943
Loss: 0.2092

In [42]:
input_text = "Supervised Learning"
input_tokens = tokenizer(input_text, return_tensors='pt')
output_tokens = model.generate(input_tokens['input_ids'], max_length=25, pad_token_id=tokenizer.pad_token_id)
output_text = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print("Response:", output_text)

Response: Supervised Learning We are given input samples (X) of a function y = f(X). We represent the entire data
